In [1]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, FunctionTransformer, OneHotEncoder
from sklearn.compose import ColumnTransformer
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
df = pd.read_csv('real_estate_value.csv')

In [3]:
df = df.dropna()

In [4]:
y = df['UnitPrice']
x = df.drop('UnitPrice', axis=1)

In [5]:
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.25, random_state=42)

In [6]:
symmetric_columns = []
skewed_columns = []

for column in x.columns:
    if df[column].skew() > -1 and df[column].skew() < 1:
        symmetric_columns.append(column)
    else:
        skewed_columns.append(column)

In [7]:
print(symmetric_columns)
print(skewed_columns)

['HouseAge', 'NoOfStores', 'Latitude']
['DistanceToMRT', 'Longitude']


In [8]:
symmetric_pipeline = Pipeline([
  ('imputer', SimpleImputer(strategy='mean')),
  ('scaler', StandardScaler())
])

skewed_pipeline = Pipeline([
  ('imputer', SimpleImputer(strategy='mean')),
  ('log_transform', FunctionTransformer(np.log1p)),
  ('scaler', StandardScaler())
])
preprocessor = ColumnTransformer([
    ('numeric_symmetric', symmetric_pipeline, symmetric_columns),
    ('numeric_skewed', skewed_pipeline, skewed_columns),
])

pipeline = Pipeline([
    ('preprocessor', preprocessor)
])

In [9]:
pipeline.fit(df)
train_transformed = pipeline.transform(train_x)
test_transformed = pipeline.transform(test_x)

In [10]:
decision_tree = DecisionTreeRegressor(random_state=42)
decision_tree.fit(train_transformed, train_y)
y_test_pred_dt = decision_tree.predict(test_transformed)

mse_dt = mean_squared_error(test_y, y_test_pred_dt)
r2_dt = r2_score(test_y, y_test_pred_dt)

In [11]:
random_forest = RandomForestRegressor(random_state=42)
random_forest.fit(train_transformed, train_y)
y_test_pred_rf = random_forest.predict(test_transformed)

mse_rf = mean_squared_error(test_y, y_test_pred_rf)
r2_rf = r2_score(test_y, y_test_pred_rf)

In [12]:
print("Mean Squared Error (Decision Tree):", mse_dt)
print("R-squared (Decision Tree):", r2_dt)
print("Mean Squared Error (Random Forest):", mse_rf)
print("R-squared (Random Forest):", r2_rf)

Mean Squared Error (Decision Tree): 74.34548076923078
R-squared (Decision Tree): 0.5312940832173635
Mean Squared Error (Random Forest): 45.04920210294844
R-squared (Random Forest): 0.7159904360894592
